# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments   Customer ShipmentStatus
0  Feb  2 2023  1:12PM  256544        12  HH-39716  Hush Hush       Released
1  Feb  2 2023  1:12PM  256544        12  HH-39720  Hush Hush       Released
2  Feb  2 2023  1:12PM  256544        12  HH-39721  Hush Hush       Released
3  Feb  2 2023  1:12PM  256544        12  HH-39722  Hush Hush       Released
4  Feb  2 2023  1:12PM  256544        12  HH-39723  Hush Hush       Released
5  Feb  2 2023  1:12PM  256544        12  HH-39724  Hush Hush       Released
6  Feb  2 2023  1:12PM  256544        12  HH-39725  Hush Hush       Released
7  Feb  2 2023  1:12PM  256544        12  HH-39726  Hush Hush       Released
8  Feb  2 2023  1:12PM  256544        12  HH-39727  Hush Hush       Released
9  Feb  2 2023  1:12PM  256544        12  HH-39728  Hush Hush       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
47  256540       Released          1
48  256541       Released          1
49  256542       Released          2
50  256543       Released         11
51  256544       Released         14

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
256540                NaN        NaN       1.0
256541                NaN        NaN       1.0
256542                NaN        NaN       2.0
256543                NaN        NaN      11.0
256544                NaN        NaN      14.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256418                3.0        5.0       3.0
256427                0.0        0.0       1.0
256440                0.0        0.0       6.0
256444                0.0        0.0      10.0
256447                0.0        0.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256418                  3          5         3
256427                  0          0         1
256440                  0          0         6
256444                  0          0        10
256447                  0          0         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               256418          3          5         3
1               256427          0          0         1
2               256440          0          0         6
3               256444          0          0        10
4               256447          0          0         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               256418         3         5        3
1               256427                            1
2               256440                            6
3               256444                           10
4               256447                            2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0   Feb  2 2023  1:12PM  256544        12              Hush Hush
14  Feb  2 2023  1:11PM  256543        10      ISDIN Corporation
25  Feb  2 2023  1:09PM  256542        10       Methapharm, Inc.
27  Feb  2 2023  1:08PM  256541        10      ISDIN Corporation
28  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd
84  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.
85  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.
86  Feb  2 2023 12:39PM  256534        16  ACG North America LLC
87  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.
88  Feb  2 2023 12:38PM  256537        21      NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Feb  2 2023  1:12PM  256544        12              Hush Hush             
1  Feb  2 2023  1:11PM  256543        10      ISDIN Corporation             
2  Feb  2 2023  1:09PM  256542        10       Methapharm, Inc.             
3  Feb  2 2023  1:08PM  256541        10      ISDIN Corporation             
4  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd             
5  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.             
6  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.             
7  Feb  2 2023 12:39PM  256534        16  ACG North America LLC             
8  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.             
9  Feb  2 2023 12:38PM  256537        21      NBTY Global, Inc.             

  Executing Released  
0                 14  
1                 11  
2                  2  
3                  1  
4        12       44  
5                  1  
6                  1  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Feb  2 2023  1:12PM  256544        12              Hush Hush       14   
1  Feb  2 2023  1:11PM  256543        10      ISDIN Corporation       11   
2  Feb  2 2023  1:09PM  256542        10       Methapharm, Inc.        2   
3  Feb  2 2023  1:08PM  256541        10      ISDIN Corporation        1   
4  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd       44   
5  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.        1   
6  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.        1   
7  Feb  2 2023 12:39PM  256534        16  ACG North America LLC        1   
8  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.        1   
9  Feb  2 2023 12:38PM  256537        21      NBTY Global, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4        12            
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Feb  2 2023  1:12PM  256544        12            Hush Hush       14   
1  Feb  2 2023  1:11PM  256543        10    ISDIN Corporation       11   
2  Feb  2 2023  1:09PM  256542        10     Methapharm, Inc.        2   
3  Feb  2 2023  1:08PM  256541        10    ISDIN Corporation        1   
4  Feb  2 2023 12:45PM  256527        19  AdvaGen Pharma, Ltd       44   

  Executing Completed  
0                      
1                      
2                      
3                      
4        12

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Feb  2 2023  1:12PM  256544        12            Hush Hush      14.0   
1  Feb  2 2023  1:11PM  256543        10    ISDIN Corporation      11.0   
2  Feb  2 2023  1:09PM  256542        10     Methapharm, Inc.       2.0   
3  Feb  2 2023  1:08PM  256541        10    ISDIN Corporation       1.0   
4  Feb  2 2023 12:45PM  256527        19  AdvaGen Pharma, Ltd      44.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4       12.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Feb  2 2023  1:12PM  256544        12            Hush Hush      14.0   
1  Feb  2 2023  1:11PM  256543        10    ISDIN Corporation      11.0   
2  Feb  2 2023  1:09PM  256542        10     Methapharm, Inc.       2.0   
3  Feb  2 2023  1:08PM  256541        10    ISDIN Corporation       1.0   
4  Feb  2 2023 12:45PM  256527        19  AdvaGen Pharma, Ltd      44.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4       12.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4873459      75.0        7.0        3.0
12         1282546      17.0        1.0        0.0
15         1025878      23.0       17.0       19.0
16          256534       1.0        0.0        0.0
19         1282451      48.0       12.0        0.0
20          256519       1.0        0.0        0.0
21         2052031       7.0        1.0        0.0
22          769566       3.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4873459      75.0        7.0        3.0
1        12  1282546      17.0        1.0        0.0
2        15  1025878      23.0       17.0       19.0
3        16   256534       1.0        0.0        0.0
4        19  1282451      48.0       12.0        0.0
5        20   256519       1.0        0.0        0.0
6        21  2052031       7.0        1.0        0.0
7        22   769566       3.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      75.0        7.0        3.0
1        12      17.0        1.0        0.0
2        15      23.0       17.0       19.0
3        16       1.0        0.0        0.0
4        19      48.0       12.0        0.0
5        20       1.0        0.0        0.0
6        21       7.0        1.0        0.0
7        22       3.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   75.0
1        12  Released   17.0
2        15  Released   23.0
3        16  Released    1.0
4        19  Released   48.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12    15   16    19   20   21   22
Status                                               
Completed   3.0   0.0  19.0  0.0   0.0  0.0  0.0  0.0
Executing   7.0   1.0  17.0  0.0  12.0  0.0  1.0  0.0
Released   75.0  17.0  23.0  1.0  48.0  1.0  7.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12    15   16    19   20   21   22
0          Completed   3.0   0.0  19.0  0.0   0.0  0.0  0.0  0.0
1          Executing   7.0   1.0  17.0  0.0  12.0  0.0  1.0  0.0
2           Released  75.0  17.0  23.0  1.0  48.0  1.0  7.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12    15   16    19   20   21   22
0  Completed   3.0   0.0  19.0  0.0   0.0  0.0  0.0  0.0
1  Executing   7.0   1.0  17.0  0.0  12.0  0.0  1.0  0.0
2   Released  75.0  17.0  23.0  1.0  48.0  1.0  7.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()